# Trader Behavior vs Market Sentiment Analysis

This notebook analyzes the relationship between **Bitcoin market sentiment (Fear & Greed Index)** and **trader performance** using historical Hyperliquid trade data.

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

## 2. Load Datasets

In [ ]:
trades = pd.read_csv('historical_data.csv')
sentiment = pd.read_csv('fear_greed_index.csv')
trades.head()

## 3. Data Cleaning & Preprocessing

In [ ]:
trades['Timestamp'] = pd.to_datetime(trades['Timestamp'], unit='ms')
trades['date'] = trades['Timestamp'].dt.date

sentiment['date'] = pd.to_datetime(sentiment['date']).dt.date
sentiment = sentiment[['date', 'classification']]
sentiment.rename(columns={'classification': 'sentiment'}, inplace=True)

trades[['Timestamp', 'date']].head()

## 4. Merge Trader Data with Market Sentiment

In [ ]:
df = trades.merge(sentiment, on='date', how='inner')
df.head()

## 5. Feature Engineering

In [ ]:
df['pnl_category'] = np.where(df['Closed PnL'] > 0, 'Profit', 'Loss')
df['trade_direction'] = df['Side'].str.capitalize()
df['sentiment_binary'] = df['sentiment'].apply(lambda x: 'Fear' if 'Fear' in x else 'Greed')

df[['Closed PnL', 'pnl_category', 'trade_direction', 'sentiment_binary']].head()

## 6. Trader Performance vs Market Sentiment

In [ ]:
performance = df.groupby('sentiment_binary')['Closed PnL'].agg(
    avg_pnl='mean', median_pnl='median', total_trades='count')

win_rate = df.groupby('sentiment_binary')['pnl_category'].apply(
    lambda x: (x == 'Profit').mean() * 100)

performance['win_rate_%'] = win_rate
performance

## 7. PnL Distribution

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='sentiment_binary', y='Closed PnL')
plt.title('PnL Distribution by Market Sentiment')
plt.show()

## 8. Trade Direction Bias

In [ ]:
direction_dist = pd.crosstab(df['sentiment_binary'], df['trade_direction'], normalize='index') * 100
direction_dist

## 9. Account-Level Performance

In [ ]:
account_perf = df.groupby(['Account', 'sentiment_binary'])['Closed PnL'].sum().unstack()
account_perf.head()

## 10. Save Final Dataset

In [ ]:
df.to_csv('merged_trader_sentiment_data.csv', index=False)